In [ ]:
import os, sys
import pandas as pd
import plotly.graph_objects as go
import io
from dateutil import parser
import scipy


In [ ]:
def change_col_to_point(x):
    if type(x) is str:
        x = x.replace(',', '.')
        return float(x)
    else:
        return float(x)

In [ ]:
path = os.getcwd()
dirs = os.listdir( path +'/Data/RTE')
dirs.sort()
i = 0
df_tot = []
for file in dirs:
    print(file)
    df = pd.read_csv('Data/RTE/'+file,encoding="ISO-8859-1", sep='\t', low_memory=False, index_col=0, header = [0,1])
    df= df.replace("*", 0)
    df = df.applymap(change_col_to_point)
    df_tot.append(df)
df = pd.concat(df_tot)

In [ ]:
df

In [ ]:
df_group_resourse = df.groupby(level=0, axis=1).sum()

In [ ]:
df.info()

In [ ]:
df_group_resourse

In [ ]:
df_group_resourse.index = [i.split('-')[0] for i in df_group_resourse.index]
    #df['Date_time'] =  pd.to_datetime(df['Date_time'], format='%Y-%m-%d-%H:%M')

In [ ]:
df_group_resourse.index = pd.to_datetime(df_group_resourse.index, format='%d/%m/%Y %H:%M')
df_group_resourse

In [ ]:
df_group_resourse['Nucléaire']

In [ ]:
start ='2018-01-01 00:00:00'
end ='2023-04-25 00:00:00'
df_group_resourse_day = pd.DataFrame(df_group_resourse,
    index=pd.date_range(start, end, freq="M")
)

In [ ]:
df_group_resourse_day['Fioul'].plot()

In [ ]:
df_group_resourse_day['Gaz'].plot()

In [ ]:
df_group_resourse_day['Nucléaire'].plot()

In [ ]:
df_group_resourse.Gaz.rolling(700, win_type ='triang').sum().plot()
df_group_resourse.Nucléaire.rolling(700, win_type ='triang').sum().plot()
df_group_resourse.Fioul.rolling(700, win_type ='triang').sum().plot()

In [ ]:
series_list = []
list_resource =["Gaz", "Nucléaire", "Fioul", "Charbon"] 
for resourse in list_resource:
    series = df_group_resourse[resourse].rolling(700, win_type ='triang').sum()
    series_list.append(series)
    


In [ ]:
def list_series_data(df, list_label):
    for resourse in list_label:
        series = df[resourse].rolling(700, win_type ='triang').sum()
        series_list.append(series)
    return series_list

In [ ]:
list_series = list_series_data(df_group_resourse, list_resource)
list_series

In [ ]:
def plot_subplot_resources(list_series,list_label):
    fig = make_subplots(rows=len(list_series), cols=1)
    
    for i in range(len(list_series)):
        fig.append_trace(go.Scatter(
            x=list_series[i].index,
            y=list_series[i],
            name =list_label[i],
        ), row=i+1, col=1)
    fig.add_vrect(x0='2020-03-11', x1='2022-02-24', line_width=0, fillcolor="red", opacity=0.1, annotation_text='Crise liée au COVID-19', annotation_position='inside top left')
    fig.add_vrect(x0='2022-02-24', x1='2023-04-25', line_width=0, fillcolor="blue", opacity=0.1, annotation_text='Guerre en Ukraine', annotation_position='inside top left')
    fig.update_layout(height=1200, width=800, title_text="Electricitè pour la production de different resourse")
    fig.show()

In [ ]:
plot_subplot_resources(series_list,list_resource)

In [ ]:
df_Gaz = df_group_resourse.Gaz.rolling(700, win_type ='triang').sum()
df_nucleaire = df_group_resourse.Nucléaire.rolling(700, win_type ='triang').sum()
df_fioul = df_group_resourse.Fioul.rolling(700, win_type ='triang').sum()
df_charbon =df_group_resourse.Charbon.rolling(700, win_type ='triang').sum()

In [ ]:
df_charbon.index[-1]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1)

fig.append_trace(go.Scatter(
    x=df_Gaz.index,
    y=df_Gaz,
    name ="Gaz",
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=df_nucleaire.index,
    y=df_nucleaire,
    name ="Nucleaire",
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=df_fioul.index,
    y=df_fioul,
    name ="Fioul",
), row=3, col=1)

fig.append_trace(go.Scatter(
    x=df_charbon.index,
    y=df_charbon,
    name ="Charbon",
), row=4, col=1)

fig.add_vrect(x0='2020-03-11', x1='2022-02-24', line_width=0, fillcolor="red", opacity=0.1, annotation_text='Crise liée au COVID-19', annotation_position='inside top left')
fig.add_vrect(x0='2022-02-24', x1='2023-04-01', line_width=0, fillcolor="blue", opacity=0.1, annotation_text='Guerre en Ukraine', annotation_position='inside top left')
fig.update_layout(height=1200, width=800, title_text="Electricitè pour la production de different resourse")
fig.show()